In [1]:
from keras.datasets import mnist
from keras.layers import MaxPooling2D
#download mnist data and split into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

import matplotlib.pyplot as plt
import numpy as np
import time
from keras.preprocessing.image import save_img
from keras.applications import vgg16
from keras import backend as K
#plot the first image in the dataset
plt.imshow(X_train[0])

X_train[0].shape

X_train = X_train.reshape(60000,28,28,1)
X_test = X_test.reshape(10000,28,28,1)

from keras.utils import to_categorical
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_train[0]
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, (5,5), activation='relu', input_shape=(28,28,1), name='conv1'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32, (5,5), activation='relu', name='conv2'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3)

model.predict(X_test[:4])
y_test[:4]

Using TensorFlow backend.


Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 138s 2ms/step - loss: 14.4861 - acc: 0.1010 - val_loss: 14.4918 - val_acc: 0.1009
Epoch 2/3
60000/60000 [==============================] - 143s 2ms/step - loss: 14.5200 - acc: 0.0992 - val_loss: 14.4918 - val_acc: 0.1009
Epoch 3/3
60000/60000 [==============================] - 138s 2ms/step - loss: 14.5200 - acc: 0.0992 - val_loss: 14.4918 - val_acc: 0.1009


array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [2]:
from keras import applications

model.summary()

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 24, 24, 64)        1664      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 8, 8, 32)          51232     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                10250     
Total para

In [4]:
input_img = model.input

In [5]:
layer_dict = dict([(layer.name, layer) for layer in model.layers[0:]])

In [6]:
def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + K.epsilon())

In [7]:
kept_filters = []
for filter_index in range(64):
    # we only scan through the first 10 filters,
    # but there are actually 64 of them
    print('Processing filter %d' % filter_index)
    start_time = time.time()
    
    layer_output = layer_dict['conv1'].output
    if K.image_data_format() == 'channels_first':
        loss = K.sum(layer_output[:, filter_index, :, :])
    else:
        loss = K.sum(layer_output[:, :, :, filter_index])
    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]

    # normalization trick: we normalize the gradient
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img], [loss, grads])
    

    # step size for gradient ascent
    step = 0.2

    # we start from a gray image with some random noise
    if K.image_data_format() == 'channels_first':
        input_img_data = np.random.random((1, 1, 28, 28))  #28*28image
    else:
        input_img_data = np.random.random((1, 28, 28, 1))
    input_img_data = (input_img_data - 0.5) * 20 + 128
    
    # we run gradient ascent for 20 steps
    for i in range(1000):
        
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step

        print('Current loss value:', loss_value)
        if loss_value <= 0.:
            # some filters get stuck to 0, we can skip them
            break
     # decode the resulting input image
    if loss_value > 0:
        img = input_img_data[0]
        kept_filters.append((img, loss_value))
    end_time = time.time()
    print('Filter %d processed in %ds' % (filter_index, end_time - start_time))

Processing filter 0
Current loss value: 0.0
Filter 0 processed in 0s
Processing filter 1
Current loss value: 0.0
Filter 1 processed in 0s
Processing filter 2
Current loss value: 34892.61
Current loss value: 34954.414
Current loss value: 35016.227
Current loss value: 35078.04
Current loss value: 35139.844
Current loss value: 35201.656
Current loss value: 35263.47
Current loss value: 35325.277
Current loss value: 35387.086
Current loss value: 35448.9
Current loss value: 35510.703
Current loss value: 35572.52
Current loss value: 35634.33
Current loss value: 35696.133
Current loss value: 35757.945
Current loss value: 35819.758
Current loss value: 35881.566
Current loss value: 35943.375
Current loss value: 36005.188
Current loss value: 36067.0
Current loss value: 36128.81
Current loss value: 36190.62
Current loss value: 36252.43
Current loss value: 36314.242
Current loss value: 36376.047
Current loss value: 36437.86
Current loss value: 36499.67
Current loss value: 36561.484
Current loss val

Current loss value: 54301.008
Current loss value: 54362.816
Current loss value: 54424.63
Current loss value: 54486.438
Current loss value: 54548.25
Current loss value: 54610.062
Current loss value: 54671.867
Current loss value: 54733.68
Current loss value: 54795.49
Current loss value: 54857.297
Current loss value: 54919.11
Current loss value: 54980.918
Current loss value: 55042.727
Current loss value: 55104.54
Current loss value: 55166.348
Current loss value: 55228.16
Current loss value: 55289.973
Current loss value: 55351.78
Current loss value: 55413.59
Current loss value: 55475.4
Current loss value: 55537.21
Current loss value: 55599.016
Current loss value: 55660.83
Current loss value: 55722.64
Current loss value: 55784.453
Current loss value: 55846.26
Current loss value: 55908.08
Current loss value: 55969.88
Current loss value: 56031.688
Current loss value: 56093.5
Current loss value: 56155.312
Current loss value: 56217.117
Current loss value: 56278.93
Current loss value: 56340.742


Current loss value: 74142.08
Current loss value: 74203.89
Current loss value: 74265.7
Current loss value: 74327.51
Current loss value: 74389.33
Current loss value: 74451.125
Current loss value: 74512.945
Current loss value: 74574.75
Current loss value: 74636.56
Current loss value: 74698.36
Current loss value: 74760.18
Current loss value: 74821.99
Current loss value: 74883.805
Current loss value: 74945.61
Current loss value: 75007.42
Current loss value: 75069.23
Current loss value: 75131.05
Current loss value: 75192.86
Current loss value: 75254.664
Current loss value: 75316.47
Current loss value: 75378.29
Current loss value: 75440.09
Current loss value: 75501.91
Current loss value: 75563.7
Current loss value: 75625.53
Current loss value: 75687.33
Current loss value: 75749.14
Current loss value: 75810.95
Current loss value: 75872.766
Current loss value: 75934.57
Current loss value: 75996.38
Current loss value: 76058.2
Current loss value: 76120.0
Current loss value: 76181.82
Current loss 

Current loss value: 93550.484
Current loss value: 93612.29
Current loss value: 93674.11
Current loss value: 93735.914
Current loss value: 93797.72
Current loss value: 93859.53
Current loss value: 93921.34
Current loss value: 93983.15
Current loss value: 94044.96
Current loss value: 94106.766
Current loss value: 94168.58
Current loss value: 94230.41
Current loss value: 94292.2
Current loss value: 94354.016
Current loss value: 94415.82
Current loss value: 94477.64
Current loss value: 94539.45
Current loss value: 94601.25
Current loss value: 94663.06
Current loss value: 94724.87
Current loss value: 94786.69
Current loss value: 94848.5
Current loss value: 94910.305
Current loss value: 94972.12
Current loss value: 95033.92
Current loss value: 95095.734
Current loss value: 95157.55
Current loss value: 95219.34
Current loss value: 95281.17
Current loss value: 95342.97
Current loss value: 95404.79
Current loss value: 95466.59
Current loss value: 95528.41
Current loss value: 95590.22
Current lo

Current loss value: 35715.766
Current loss value: 35758.47
Current loss value: 35801.164
Current loss value: 35843.87
Current loss value: 35886.57
Current loss value: 35929.273
Current loss value: 35971.97
Current loss value: 36014.68
Current loss value: 36057.375
Current loss value: 36100.08
Current loss value: 36142.78
Current loss value: 36185.477
Current loss value: 36228.184
Current loss value: 36270.883
Current loss value: 36313.582
Current loss value: 36356.28
Current loss value: 36398.984
Current loss value: 36441.688
Current loss value: 36484.387
Current loss value: 36527.09
Current loss value: 36569.79
Current loss value: 36612.492
Current loss value: 36655.195
Current loss value: 36697.89
Current loss value: 36740.598
Current loss value: 36783.297
Current loss value: 36826.0
Current loss value: 36868.703
Current loss value: 36911.402
Current loss value: 36954.105
Current loss value: 36996.805
Current loss value: 37039.508
Current loss value: 37082.207
Current loss value: 371

...

Current loss value: 15324.432
Current loss value: 15342.51
Current loss value: 15360.588
Current loss value: 15378.666
Current loss value: 15396.744
Current loss value: 15414.823
Current loss value: 15432.9
Current loss value: 15450.978
Current loss value: 15469.057
Current loss value: 15487.136
Current loss value: 15505.215
Current loss value: 15523.293
Current loss value: 15541.371
Current loss value: 15559.449
Current loss value: 15577.525
Current loss value: 15595.6045
Current loss value: 15613.683
Current loss value: 15631.762
Current loss value: 15649.84
Current loss value: 15667.918
Current loss value: 15685.996
Current loss value: 15704.074
Current loss value: 15722.15
Current loss value: 15740.23
Current loss value: 15758.309
Current loss value: 15776.387
Current loss value: 15794.465
Current loss value: 15812.543
Current loss value: 15830.623
Current loss value: 15848.699
Current loss value: 15866.777
Current loss value: 15884.855
Current loss value: 15902.934
Current loss va

Current loss value: 22465.326
Current loss value: 22483.402
Current loss value: 22501.484
Current loss value: 22519.562
Current loss value: 22537.64
Current loss value: 22555.715
Current loss value: 22573.793
Current loss value: 22591.873
Current loss value: 22609.953
Current loss value: 22628.03
Current loss value: 22646.11
Current loss value: 22664.188
Current loss value: 22682.266
Current loss value: 22700.342
Current loss value: 22718.42
Current loss value: 22736.5
Current loss value: 22754.578
Current loss value: 22772.656
Current loss value: 22790.734
Current loss value: 22808.812
Current loss value: 22826.889
Current loss value: 22844.969
Current loss value: 22863.045
Current loss value: 22881.125
Current loss value: 22899.203
Current loss value: 22917.281
Current loss value: 22935.36
Current loss value: 22953.438
Current loss value: 22971.516
Current loss value: 22989.594
Current loss value: 23007.672
Current loss value: 23025.75
Current loss value: 23043.83
Current loss value:

Current loss value: 3208.4556
Current loss value: 3218.159
Current loss value: 3227.8623
Current loss value: 3237.566
Current loss value: 3247.2695
Current loss value: 3256.973
Current loss value: 3266.676
Current loss value: 3276.38
Current loss value: 3286.0835
Current loss value: 3295.7866
Current loss value: 3305.4902
Current loss value: 3315.1936
Current loss value: 3324.897
Current loss value: 3334.6008
Current loss value: 3344.3037
Current loss value: 3354.0073
Current loss value: 3363.711
Current loss value: 3373.4146
Current loss value: 3383.118
Current loss value: 3392.8213
Current loss value: 3402.525
Current loss value: 3412.2288
Current loss value: 3421.9316
Current loss value: 3431.6353
Current loss value: 3441.3389
Current loss value: 3451.042
Current loss value: 3460.7456
Current loss value: 3470.4492
Current loss value: 3480.1528
Current loss value: 3489.8564
Current loss value: 3499.5596
Current loss value: 3509.263
Current loss value: 3518.9663
Current loss value: 35

Current loss value: 6692.0015
Current loss value: 6701.705
Current loss value: 6711.408
Current loss value: 6721.1123
Current loss value: 6730.816
Current loss value: 6740.5186
Current loss value: 6750.2227
Current loss value: 6759.9263
Current loss value: 6769.629
Current loss value: 6779.333
Current loss value: 6789.036
Current loss value: 6798.7397
Current loss value: 6808.443
Current loss value: 6818.147
Current loss value: 6827.8496
Current loss value: 6837.553
Current loss value: 6847.257
Current loss value: 6856.9604
Current loss value: 6866.6636
Current loss value: 6876.3677
Current loss value: 6886.071
Current loss value: 6895.7744
Current loss value: 6905.4785
Current loss value: 6915.1816
Current loss value: 6924.885
Current loss value: 6934.588
Current loss value: 6944.2915
Current loss value: 6953.9956
Current loss value: 6963.699
Current loss value: 6973.403
Current loss value: 6983.1055
Current loss value: 6992.8086
Current loss value: 7002.5127
Current loss value: 7012.

Current loss value: 10427.837
Current loss value: 10437.541
Current loss value: 10447.245
Current loss value: 10456.948
Current loss value: 10466.652
Current loss value: 10476.355
Current loss value: 10486.059
Current loss value: 10495.763
Current loss value: 10505.467
Current loss value: 10515.169
Current loss value: 10524.872
Current loss value: 10534.575
Current loss value: 10544.278
Current loss value: 10553.982
Current loss value: 10563.6875
Current loss value: 10573.389
Current loss value: 10583.094
Current loss value: 10592.797
Current loss value: 10602.501
Current loss value: 10612.203
Current loss value: 10621.906
Current loss value: 10631.611
Current loss value: 10641.314
Current loss value: 10651.018
Current loss value: 10660.723
Current loss value: 10670.425
Current loss value: 10680.129
Current loss value: 10689.831
Current loss value: 10699.535
Current loss value: 10709.238
Current loss value: 10718.941
Current loss value: 10728.645
Current loss value: 10738.349
Current l

In [8]:
# we will stich the 16 filters on a 5 x 5 grid. There are some filters that gave activation with the element sum = 0.
n = 5

# the filters that have the highest loss are assumed to be better-looking.
# we will only keep the top 16 filters.
kept_filters.sort(key=lambda x: x[1], reverse=True)
kept_filters = kept_filters[:n * n]

# build a black picture with enough space for
# our  16 filters of size 28 x 28, with a 5px margin in between
margin = 5
width = n * 28 + (n - 1) * margin
height = n * 28 + (n - 1) * margin
stitched_filters = np.zeros((width, height, 1))

# fill the picture with our saved filters
for i in range(n):
    for j in range(n):
        img, loss = kept_filters[i * n + j]
        width_margin = (28 + margin) * i
        height_margin = (28 + margin) * j
        stitched_filters[
            width_margin: width_margin + 28,
            height_margin: height_margin + 28, :] = img

# save the result to disk
save_img('3stitched_filters_%dx%d.png' % (n, n), stitched_filters)


In [ ]:
#con1 filters
